<h1>Flat prices analysis in St. Petersburg</h1>

<div style="background:#abd5f5; border:1px solid #b3deff; padding: 20px">
    <h2 style="color:#002b63">Table of content</h2>
<ul>
    <li>Indroduction</li>
    <li>Data 1: web scraping</li>
    <li>Data 2: data cleaning</li>
</ul>
    </div>

<h2>Indroduction</h2>

This notebook introduces web scraping work. I will parse data from https://spb.cian.ru/. At the exit, I will have a CSV file with a list of flats saling in St. Petersburg. The file will contain a link to the page of the apartment, price, area, number of rooms, and other parameters from the site. This file can be used for the analysis of the flat market and to predict prices.

<h2>Data 1: web scraping</h2>

Firstly, I import libraries

In [1]:
import requests
from bs4 import BeautifulSoup # for web scraping
import pandas as pd
import numpy as np
import re #regexp
from ipywidgets import IntProgress #for progress bar
from IPython.display import display

Let's get HTML code from the page https://spb.cian.ru/kupit-kvartiru/. It is the start page with a flat list. 

In [2]:
req = requests.get('https://spb.cian.ru/kupit-kvartiru/')
soup = BeautifulSoup(req.text, "lxml")

Let's get all links on apartments from the page. Links start with "https://spb.cian.ru/sale/flat/".

In [3]:
all_hrefs = [a.get('href') for a in soup.find_all('a')] #list of all links
flat_hrefs=[a for a in all_hrefs if 'https://spb.cian.ru/sale/flat/' in str(a)] #filtering the list
flat_hrefs=list(set(flat_hrefs)) #removing duplicates in the list
flat_hrefs[0:5]

['https://spb.cian.ru/sale/flat/234775065/',
 'https://spb.cian.ru/sale/flat/239273301/',
 'https://spb.cian.ru/sale/flat/250966190/',
 'https://spb.cian.ru/sale/flat/249950664/',
 'https://spb.cian.ru/sale/flat/250766812/']

It is a function for getting all parameters from an apartment page.

In [4]:
def add_flats(hrefs,descr_attr):
    flats=pd.DataFrame()
    progressbar=IntProgress(min=0, max=len(hrefs), value=0) #progress bar
    display(progressbar)
    for i,href in enumerate(hrefs):
        # getting html code of the flat page
        try:
            req1 = requests.get(href)
        except requests.exceptions.RequestException as e:  
            raise SystemExit(e)
        
        soup = BeautifulSoup(req1.text, "lxml")
        # if we don't have header we stop parsing
        if soup.h1 is None:
            print('Parsing is stopped')
            break
        flats.loc[i,'link']=href
        flats.loc[i,'head']=soup.h1.text # header contains the number of rooms
        flats.loc[i,'addr']=soup.address.text # getting the address
        flats.loc[i,'price']=soup.find(attrs= {"itemprop":"price"}).text #getting the price
        
        #getting distances from nearby metro stations
        metros=[t.text for t in soup.find_all(attrs= {"data-name":"renderUnderground"})]
        for k,val in enumerate(metros):
            flats.loc[i,'metro'+str(k)]=val
            
        #getting description which contains the square, the floor and the year of built 
        descr=str(soup.find(attrs= {"data-name":"Description"}))
        while '">' in descr:
            if (descr.find('">')+2)<descr.find('<'):
                if descr[descr.find('">')+2:descr.find('<')] in descr_attr:
                    flats.loc[i,descr[descr.find('">')+2:descr.find('<')]]=t
                t=descr[descr.find('">')+2:descr.find('<')]
            descr=descr[descr.find('<')+2:]
            
        #getting features
        label_info=[t.span.text for t in soup.find_all(attrs= {"data-name":"AdditionalFeatureItem"})]
        info=[t.text for t in soup.find_all(attrs= {"data-name":"AdditionalFeatureItem"})]
        for val,label in zip(info,label_info):
            flats.loc[i,label]=val[val.find(label)+len(label):]
        progressbar.value = i+1
    return flats

Let's get flats from the first page

In [5]:
descr_attr=['Общая','Жилая','Кухня','Этаж','Срок сдачи','Построен'] 
flats=add_flats(flat_hrefs,descr_attr)
flats.head()

IntProgress(value=0, max=28)

,link,head,addr,price,metro0,metro1,metro2,Общая,Жилая,Кухня,...,Срок сдачи,Тип жилья,Высота потолков,Санузел,Балкон/лоджия,Отделка,Построен,Планировка,Ремонт,Вид из окон
0,https://spb.cian.ru/sale/flat/234775065/,"2-комн. квартира, 74,13 м²","Санкт-Петербург, р-н Приморский, Юнтолово, Пла...",13 143 249 ₽,Комендантский проспект ⋅ 6 мин. на транспорте,Беговая ⋅ 7 мин. на транспорте,Старая Деревня ⋅ 8 мин. на транспорте,"74,13 м²","26,3 м²",22 м²,...,3 кв. 2022,Новостройка,"3,3 м","1 совмещенный, 1 раздельный",1 лоджия,NaN,NaN,NaN,NaN,NaN
1,https://spb.cian.ru/sale/flat/239273301/,"1-комн. квартира, 47,81 м²","Санкт-Петербург, р-н Петроградский, Посадский,...",12 430 600 ₽,Петроградская ⋅ 13 мин. пешком,Выборгская ⋅ 17 мин. пешком,Горьковская ⋅ 20 мин. пешком,"47,81 м²",NaN,NaN,...,1 кв. 2021,Новостройка,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://spb.cian.ru/sale/flat/250966190/,"1-комн. апартаменты, 44,1 м²","Санкт-Петербург, р-н Курортный, мкр. Сестрорец...",13 800 000 ₽,Беговая ⋅ 29 мин. на транспорте,Комендантский проспект ⋅ 31 мин. на транспорте,Зенит ⋅ 32 мин. на транспорте,"44,1 м²","14,1 м²","19,4 м²",...,4 кв. 2020,Новостройка Апартаменты,NaN,NaN,NaN,Чистовая,NaN,NaN,NaN,NaN
3,https://spb.cian.ru/sale/flat/249950664/,"2-комн. квартира, 56,29 м²","Санкт-Петербург, р-н Приморский, Юнтолово, Нью...",8 730 851 ₽,Комендантский проспект ⋅ 6 мин. на транспорте,Беговая ⋅ 7 мин. на транспорте,Старая Деревня ⋅ 9 мин. на транспорте,"56,29 м²","29,3 м²","10,9 м²",...,3 кв. 2021,Новостройка,NaN,2 раздельных,2 лоджии,NaN,NaN,NaN,NaN,NaN
4,https://spb.cian.ru/sale/flat/250766812/,"2-комн. квартира, 70,8 м²","Санкт-Петербург, р-н Приморский, Комендантский...",13 650 000 ₽,Пионерская ⋅ 13 мин. пешком,Комендантский проспект ⋅ 3 мин. на транспорте,Удельная ⋅ 5 мин. на транспорте,"70,8 м²",NaN,10 м²,...,NaN,Вторичка,"2,8 м",1 раздельный,1 лоджия,NaN,2006,Изолированная,Евроремонт,На улицу и двор


Let's add all flats into flats_all. It is a dataframe where I will collect all flats.

In [6]:
flats_all=pd.DataFrame()
flats_all=flats_all.append(flats,ignore_index=True)
flats_all.shape

(28, 21)

Let's go through all pages.

In [7]:
n_page=1
while n_page<=54:
    print('page number:',str(n_page))
    #request to cian
    try:
        req = requests.get('https://spb.cian.ru/cat.php?deal_type=sale&engine_version=2&offer_type=flat&p='
                           +str(n_page)+'&region=2')
    except requests.exceptions.RequestException as e:  
        raise SystemExit(e)

    soup = BeautifulSoup(req.text, "lxml")
    # if we don't have header we stop parsing
    if soup.h1 is None:
        print('Parsing is stopped')
        break
    #getting all links from the page
    all_hrefs = [a.get('href') for a in soup.find_all('a')]
    flat_hrefs=[a for a in all_hrefs if 'https://spb.cian.ru/sale/flat/' in str(a)]
    #getting all parameters from an apartment page.
    flat_hrefs=list(set(flat_hrefs))
    flats=add_flats(flat_hrefs,descr_attr)
    flats_all=flats_all.append(flats,ignore_index=True)
    print('number of rows',flats_all.shape[0])
    n_page+=1

page number: 1


IntProgress(value=0, max=28)

number of rows 56
page number: 2


IntProgress(value=0, max=28)

number of rows 84
page number: 3


...

number of rows 1484
page number: 53


IntProgress(value=0, max=28)

number of rows 1512
page number: 54


IntProgress(value=0, max=25)

number of rows 1537


Let's check the counts of rows and columns in the result dataframe.

In [8]:
flats_all.shape

(1537, 21)

In [9]:
flats_all.tail()

,link,head,addr,price,metro0,metro1,metro2,Общая,Жилая,Кухня,...,Срок сдачи,Тип жилья,Высота потолков,Санузел,Балкон/лоджия,Отделка,Построен,Планировка,Ремонт,Вид из окон
1532,https://spb.cian.ru/sale/flat/250580748/,"2-комн. квартира, 55,02 м²","Санкт-Петербург, р-н Приморский, Лахта-Ольгино...",6 766 360 ₽,Беговая ⋅ 9 мин. на транспорте,Комендантский проспект ⋅ 11 мин. на транспорте,Зенит ⋅ 12 мин. на транспорте,"55,02 м²","26,1 м²","17,9 м²",...,2 кв. 2023,Новостройка,"2,6 м",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1533,https://spb.cian.ru/sale/flat/250580813/,"2-комн. квартира, 54,87 м²","Санкт-Петербург, р-н Приморский, Лахта-Ольгино...",6 790 163 ₽,Беговая ⋅ 9 мин. на транспорте,Комендантский проспект ⋅ 11 мин. на транспорте,Зенит ⋅ 12 мин. на транспорте,"54,87 м²","21,2 м²","23,3 м²",...,2 кв. 2023,Новостройка,"2,6 м",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1534,https://spb.cian.ru/sale/flat/250213237/,"1-комн. квартира, 31,6 м²","Санкт-Петербург, р-н Приморский, № 65, ул. Опт...",7 500 000 ₽,Беговая,Комендантский проспект,Старая Деревня,"31,6 м²","14,3 м²","8,1 м²",...,NaN,Вторичка,NaN,1 раздельный,NaN,NaN,2019,NaN,Дизайнерский,NaN
1535,https://spb.cian.ru/sale/flat/249798150/,"2-комн. квартира, 55,4 м²","Санкт-Петербург, р-н Приморский, Коломяги, Орл...",7 756 000 ₽,Проспект Просвещения ⋅ 5 мин. на транспорте,Озерки ⋅ 6 мин. на транспорте,Парнас ⋅ 9 мин. на транспорте,"55,4 м²","27,5 м²","13,7 м²",...,3 кв. 2021,Новостройка,"2,6 м",1 раздельный,NaN,Чистовая,NaN,NaN,NaN,На улицу
1536,https://spb.cian.ru/sale/flat/244365478/,"1-комн. квартира, 24,27 м²","Санкт-Петербург, р-н Приморский, Юнтолово, Пла...",3 754 569 ₽,Комендантский проспект ⋅ 6 мин. на транспорте,Беговая ⋅ 7 мин. на транспорте,Старая Деревня ⋅ 8 мин. на транспорте,"24,27 м²","17,8 м²",NaN,...,2 кв. 2022,Новостройка,NaN,1 совмещенный,1 лоджия,NaN,NaN,NaN,NaN,NaN


<h2>Data 2: data cleaning</h2>

Let's rename fields and check the structure

In [10]:
flats_all=flats_all.rename(columns={'Общая':'total_area','Жилая':'living_area','Кухня':'kitchen_area','Этаж':'floor'})
flats_all=flats_all.rename(columns={'Срок сдачи':'deadline','Построен':'year_of_construction','Тип жилья':'type'})
flats_all=flats_all.rename(columns={'Вид из окон':'view','Отделка':'finishing','Санузел':'bathrooms','Ремонт':'renovation'})
flats_all=flats_all.rename(columns={'Высота потолков':'height','Балкон/лоджия':'balconies','Планировка':'layout'})
flats_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1537 entries, 0 to 1536
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   link                  1537 non-null   object
 1   head                  1537 non-null   object
 2   addr                  1537 non-null   object
 3   price                 1537 non-null   object
 4   metro0                1394 non-null   object
 5   metro1                905 non-null    object
 6   metro2                766 non-null    object
 7   total_area            1537 non-null   object
 8   living_area           1355 non-null   object
 9   kitchen_area          1252 non-null   object
 10  floor                 1537 non-null   object
 11  deadline              422 non-null    object
 12  type                  1537 non-null   object
 13  height                796 non-null    object
 14  bathrooms             1254 non-null   object
 15  balconies             846 non-null    

Sometimes a flat can be on different pages. Let's delete duplicates and check that number of rows become less.

In [11]:
flats_all.drop_duplicates(ignore_index=True,inplace=True)
flats_all.shape

(1414, 21)

Let's check the first 5 lines in the column "head".

In [12]:
flats_all[['head']].head()

,head
0,"2-комн. квартира, 74,13 м²"
1,"1-комн. квартира, 47,81 м²"
2,"1-комн. апартаменты, 44,1 м²"
3,"2-комн. квартира, 56,29 м²"
4,"2-комн. квартира, 70,8 м²"


As we see the column "head" contains number of rooms and dublicates the column "total_area". Let's extract number of rooms into the column "rooms" and delete column "head"

In [13]:
flats_all['rooms']=flats_all['head'].str.extract(r'(^\w+)')
flats_all.drop(['head'],axis='columns',inplace=True)
flats_all[['rooms']].head()

0    2
1    1
2    1
3    2
4    2
Name: rooms, dtype: object

Let's check a random address.

In [14]:
flats_all.loc[123,'addr']

'Санкт-Петербург, р-н Петроградский, Чкаловское, Песочная наб., 12На карте'

As we see the address contains a city, an area, a neighborhood, and a street. Let's separate all of the features and check them.

In [15]:
flats_all['city']=flats_all['addr'].str.extract(r'(.*?),')
flats_all['area']=flats_all['addr'].str.extract(r',(.*?),')
flats_all['neighborhood']=flats_all['addr'].str.extract(r',.*?,(.*?),')
flats_all['street']=flats_all['addr'].str.extract(r',.*?,.*?,(.*?)На карте')
flats_all.drop(['addr'],axis='columns',inplace=True)
print('city:',flats_all.loc[123,'city'])
print('area:',flats_all.loc[123,'area'])
print('neighborhood:',flats_all.loc[123,'neighborhood'])
print('street:',flats_all.loc[123,'street'])

city: Санкт-Петербург
area:  р-н Петроградский
neighborhood:  Чкаловское
street:  Песочная наб., 12


Let's check the 5 first prices.

In [16]:
flats_all[['price']].head()

,price
0,13 143 249 ₽
1,12 430 600 ₽
2,13 800 000 ₽
3,8 730 851 ₽
4,13 650 000 ₽


It is necessary to remove all symbols except digits and convert a string price into a float format.

In [17]:
flats_all['price']=flats_all['price'].str.replace(r'[^\d]', '')
flats_all['price']=flats_all['price'].astype(float)
flats_all[['price']].head()

,price
0,13143249.0
1,12430600.0
2,13800000.0
3,8730851.0
4,13650000.0


Let's check information about metro stations. We need the first and the nearest station.

In [18]:
flats_all[['metro0']].head()

,metro0
0,Комендантский проспект ⋅ 6 мин. на транспорте
1,Петроградская ⋅ 13 мин. пешком
2,Беговая ⋅ 29 мин. на транспорте
3,Комендантский проспект ⋅ 6 мин. на транспорте
4,Пионерская ⋅ 13 мин. пешком


The column contains a metro station name and a distance to it. We need to separate them. Also, we need to remove unnecessary columns.

In [19]:
# It is function for calculating distances to metro stations in km
#"пешком" means "on foot" (approximate speed=80 metros/min)
#"на транспорте" means "on transport" (approximate speed=330 metros/min)
def metro_distance(a):
    if 'на транспорте' in a:
        if '<1' in a:
            dist=0.1
        else:
            dist=330*float(a[a.find('⋅')+1:a.find('мин')])/1000
    elif 'пешком' in a:
        if '<1' in a:
            dist=0.05
        else:
            dist=80*float(a[a.find('⋅')+1:a.find('мин')])/1000
    else:
        dist=0
    return(dist)

#separating metro names and metro distances
flats_all['metro0']=flats_all['metro0'].astype(str)
flats_all['metro_name']=flats_all['metro0'].apply(lambda name:name[0:name.find('⋅')] if name.find('⋅')>-1 else name)
flats_all['metro_km']=flats_all['metro0'].apply(metro_distance)
flats_all.drop(['metro0','metro1','metro2'],axis='columns',inplace=True)
flats_all[['metro_name','metro_km']].head()

,metro_name,metro_km
0,Комендантский проспект,1.98
1,Петроградская,1.04
2,Беговая,9.57
3,Комендантский проспект,1.98
4,Пионерская,1.04


Let's check columns with areas and heights of flats.

In [20]:
flats_all[['total_area','living_area','kitchen_area','height']].head()

,total_area,living_area,kitchen_area,height
0,"74,13 м²","26,3 м²",22 м²,"3,3 м"
1,"47,81 м²",NaN,NaN,NaN
2,"44,1 м²","14,1 м²","19,4 м²",NaN
3,"56,29 м²","29,3 м²","10,9 м²",NaN
4,"70,8 м²",NaN,10 м²,"2,8 м"


Removing units of measure and counverting columns into float.

In [21]:
flats_all['total_area']=flats_all['total_area'].str.replace(r'[\sм²]','')
flats_all['living_area']=flats_all['living_area'].str.replace(r'[\sм²]','')
flats_all['kitchen_area']=flats_all['kitchen_area'].str.replace(r'[\sм²]','')
flats_all['total_area']=flats_all['total_area'].str.replace(r'[,]','.').astype(float)
flats_all['living_area']=flats_all['living_area'].str.replace(r'[,]','.').astype(float)
flats_all['kitchen_area']=flats_all['kitchen_area'].str.replace(r'[,]','.').astype(float)
flats_all['height']=flats_all['height'].str.replace(r'[\sм]','')
flats_all['height']=flats_all['height'].str.replace(r'[,]','.').astype(float)
flats_all[['total_area','living_area','kitchen_area','height']].head()

,total_area,living_area,kitchen_area,height
0,74.13,26.3,22.0,3.3
1,47.81,NaN,NaN,NaN
2,44.10,14.1,19.4,NaN
3,56.29,29.3,10.9,NaN
4,70.80,NaN,10.0,2.8


Let's check information about floors.

In [22]:
flats_all[['floor']].head()

,floor
0,3 из 12
1,2 из 8
2,2 из 5
3,5 из 13
4,13 из 19


Renaming and slicing the floor and the number of floors

In [23]:
floors=flats_all['floor'].str.split(' из ', n = 1, expand = True)
flats_all['floor']=floors[0]
flats_all['number_of_floors']=floors[1]
flats_all[['floor','number_of_floors']].head()

,floor,number_of_floors
0,3,12
1,2,8
2,2,5
3,5,13
4,13,19


I check the year of the deadline for new buildings, and the year of construction for the old ones. 

In [24]:
flats_all[['deadline','year_of_construction']].head()

,deadline,year_of_construction
0,3 кв. 2022,NaN
1,1 кв. 2021,NaN
2,4 кв. 2020,NaN
3,3 кв. 2021,NaN
4,NaN,2006


Now we will convert the year of deadline and the year of construction. After we replace empty values in "year_of_construction" and remove the column "deadline".

In [25]:
flats_all['deadline']=flats_all['deadline'].str.extract(r'(20.*)').astype(float)
flats_all['year_of_construction']=flats_all['year_of_construction'].combine_first(flats_all['deadline'])
flats_all.drop(columns=['deadline'],axis=1,inplace=True)
flats_all[['year_of_construction']].head()

,year_of_construction
0,2022
1,2021
2,2020
3,2021
4,2006


Next I will check the "type" column

In [26]:
flats_all[['type']].head()

,type
0,Новостройка
1,Новостройка
2,Новостройка Апартаменты
3,Новостройка
4,Вторичка


Removing all unnecessary from the "type" column

In [27]:
flats_all['type']=flats_all['type'].str.extract(r'(^\w+)')
flats_all[['type']].head()

,type
0,Новостройка
1,Новостройка
2,Новостройка
3,Новостройка
4,Вторичка


Let's check bathrooms and balconies.

In [28]:
flats_all[['bathrooms','balconies']].head()

,bathrooms,balconies
0,"1 совмещенный, 1 раздельный",1 лоджия
1,NaN,NaN
2,NaN,NaN
3,2 раздельных,2 лоджии
4,1 раздельный,1 лоджия


We need count the number of bathrooms and balconies and convert them to a float format.

In [29]:
def count_rooms(txt):
    txt=str(txt)
    txt=re.sub(r'[^\d]', '',txt)
    sumb=0
    for i in range(len(txt)):
        sumb=sumb+float(txt[i])
    return(sumb)
flats_all['bathrooms']=flats_all['bathrooms'].apply(count_rooms)
flats_all['balconies']=flats_all['balconies'].apply(count_rooms)
flats_all[['bathrooms','balconies']].head()

,bathrooms,balconies
0,2.0,1.0
1,0.0,0.0
2,0.0,0.0
3,2.0,2.0
4,1.0,1.0


Finally, I write flats_all to a CSV file for further processing. 

In [30]:
flats_all.to_csv('data/flats_all.csv',sep=';',index=False)
flats_all.head()

,link,price,total_area,living_area,kitchen_area,floor,type,height,bathrooms,balconies,...,renovation,view,rooms,city,area,neighborhood,street,metro_name,metro_km,number_of_floors
0,https://spb.cian.ru/sale/flat/234775065/,13143249.0,74.13,26.3,22.0,3,Новостройка,3.3,2.0,1.0,...,NaN,NaN,2,Санкт-Петербург,р-н Приморский,Юнтолово,"Планерная ул., 94",Комендантский проспект,1.98,12
1,https://spb.cian.ru/sale/flat/239273301/,12430600.0,47.81,NaN,NaN,2,Новостройка,NaN,0.0,0.0,...,NaN,NaN,1,Санкт-Петербург,р-н Петроградский,Посадский,"ул. Рентгена, 25",Петроградская,1.04,8
2,https://spb.cian.ru/sale/flat/250966190/,13800000.0,44.10,14.1,19.4,2,Новостройка,NaN,0.0,0.0,...,NaN,NaN,1,Санкт-Петербург,р-н Курортный,мкр. Сестрорецк,"ул. Максима Горького, 2Ас2",Беговая,9.57,5
3,https://spb.cian.ru/sale/flat/249950664/,8730851.0,56.29,29.3,10.9,5,Новостройка,NaN,2.0,2.0,...,NaN,NaN,2,Санкт-Петербург,р-н Приморский,Юнтолово,Нью Тайм жилой комплекс,Комендантский проспект,1.98,13
4,https://spb.cian.ru/sale/flat/250766812/,13650000.0,70.80,NaN,10.0,13,Вторичка,2.8,1.0,1.0,...,Евроремонт,На улицу и двор,2,Санкт-Петербург,р-н Приморский,Комендантский аэродром,"аллея Поликарпова, 6к1",Пионерская,1.04,19
